In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Tavoite

Tämän harjoituksen tarkoituksena on luoda koneoppimisen malli, joka yrittää ennustaa jääkiekon sm-liigan datan pohjalta sitä, voittaako kotijoukkue pelin vai ei. Normaalisti peli voi päättyä varsinaisella peliajalla kolmella tavalla (kotivoitto, tasapeli tai vierasvoitto), mutta yksinkertaistetaan "ongelmaa" hieman ja niputetaan vierasvoitto ja tasapeli yhdeksi lopputulemaksi.

In [126]:
df = pd.read_excel('liiga_data_uusi.xlsx')

In [127]:
df.head()

,Peli,Pvm,Aika,Kotijoukkue,Vierasjoukkue,Kotimaalit,Vierasmaalit,Tapa,Yleisöä,Peliaikatulos,Peliaikamaalit,Kausi,Merkki,Koti_pelasi_eilen,Vieras_pelasi_eilen
0,1,2008-09-11,18:30:00,HIFK,Jokerit,5,3,N,-,5-3,8,2008-2009,1,0,0
1,2,2008-09-11,18:30:00,Ilves,Tappara,5,4,N,-,5-4,9,2008-2009,1,0,0
2,3,2008-09-11,18:30:00,KalPa,JYP,1,3,N,-,1-3,4,2008-2009,2,0,0
3,4,2008-09-11,18:30:00,Kärpät,SaiPa,7,0,N,-,7-0,7,2008-2009,1,0,0
4,5,2008-09-11,18:30:00,Pelicans,Blues,1,4,N,-,1-4,5,2008-2009,2,0,0


Datassa yleisömäärät on tilastoina vasta kaudesta 2014-2015 eteenpäin. Tämän vuoksi poistetaan tätä aiemmat kaudet, koska halutaan lisätä sarakkeeksi hallin täyttöaste, jonka tarkoituksena on kuvata kotiyleisön tukea ja hallin yleistä "fiilistä" pelin aikana. 

# Datan täydennystä

In [128]:
#Otetaan mukaan vain kaudet, joista tiedetään yleisömäärät
df = df[(df.Kausi == '2014-2015') | (df.Kausi == '2015-2016') | (df.Kausi == '2016-2017') | (df.Kausi == '2017-2018')]

In [129]:
#Nollataan indeksi
df.reset_index(inplace=True)

In [130]:
df.drop(['index'],axis=1,inplace=True)

In [131]:
df.shape

(1770, 15)

Lisätään dataan sarakkeet: Kuukausi, viikonpäivä ja hallin täyttöaste

In [132]:
df['Kuukausi'] = df.Pvm.dt.month
df['Viikonpäivä'] = df.Pvm.dt.weekday_name

In [133]:
#täyttöasteen laskenta:

kapasiteetti = pd.read_excel('kapasiteetit.xlsx')

täyttöaste = []

for i in range(0,1770):
    
    joukkue = df.Kotijoukkue[i]
    aste = float(df.Yleisöä[i] / int(kapasiteetti[kapasiteetti.Kotijoukkue == joukkue].Hallikapasiteetti.values))
    täyttöaste.append(aste)
    
df['Täyttöaste'] = täyttöaste

#HUOM! Esim ottelussa HIFK-KÄRPÄT (5.12.2017) täyttöaste on lähes 2, koska kyseinen ottelu oli ulkoilmaottelu, 
#eikä sitä näin ollen pelattu Helsingin jäähallissa

In [135]:
df.head()

,Peli,Pvm,Aika,Kotijoukkue,Vierasjoukkue,Kotimaalit,Vierasmaalit,Tapa,Yleisöä,Peliaikatulos,Peliaikamaalit,Kausi,Merkki,Koti_pelasi_eilen,Vieras_pelasi_eilen,Kuukausi,Viikonpäivä,Täyttöaste
0,1,2014-09-10,18:30:00,Kärpät,Tappara,3,1,N,5608,3-1,4,2014-2015,1,0,0,9,Wednesday,0.847898
1,2,2014-09-11,18:30:00,HIFK,Blues,3,2,N,6413,3-2,5,2014-2015,1,0,0,9,Thursday,0.782073
2,3,2014-09-11,18:30:00,Lukko,Ässät,2,3,VL,4429,2-2,4,2014-2015,X,0,0,9,Thursday,0.820185
3,4,2014-09-12,18:30:00,Ilves,Tappara,1,2,N,6349,1-2,3,2014-2015,2,0,0,9,Friday,0.869726
4,5,2014-09-12,18:30:00,JYP,Sport,4,2,N,3843,4-2,6,2014-2015,1,0,0,9,Friday,0.866126


Lisätään dataan myös vierasjoukkueen kulkema matka pelipaikkakunnalle. On huomioitavaa, ettei tämä aina pidä paikkaansa, koska jos esimerkiksi Kuopion KalPa pelaa perjantaina Ässiä vastaan Porissa ja lauantaina Lukkoa vastaan Raumalla, ei joukkue todennäköisesti matkusta välillä kotiinsa Kuopioon vaan yöpyy jossakin pelipaikkakuntien välillä. Vieraspelimatkustus on kuitenkin hyvä ottaa mukaan eräänlaisena lisärasitusta kuvaavana indikaattorina. Oletettavaa myös on, että liigajoukkueet tekevät runkosarjan aikana matkansa bussilla eikä lentäen, kuten play-off peleissä välillä tehdään pitkien välimatkojen pelipareissa.

In [154]:
matkat = pd.read_excel('matkat.xlsx')

matka = []

for i in range(0,1770):
    
    väli = int(matkat[(matkat.Koti == df.iloc[i].Kotijoukkue) & (matkat.Vieras == df.iloc[i].Vierasjoukkue)].Matka.values)
    
    matka.append(väli)
    
df['Vierasmatka'] = matka

Lisätään dataan lopuksi ennustettava "label" eli voittaako kotijoukkue ottelun vai ei. 

In [160]:
kotivoitto = []

for i in range(0,1770):
    
    if df.Merkki[i] == '1':
        
        kotivoitto.append(1)
    else:
        
        kotivoitto.append(0)
    
df['Kotivoitto'] = kotivoitto   

# Datan valmistelu deep-learning mallia varten

Otetaan mukaan vain tarvittavat sarakkeet

In [161]:
df.head()

,Peli,Pvm,Aika,Kotijoukkue,Vierasjoukkue,Kotimaalit,Vierasmaalit,Tapa,Yleisöä,Peliaikatulos,Peliaikamaalit,Kausi,Merkki,Koti_pelasi_eilen,Vieras_pelasi_eilen,Kuukausi,Viikonpäivä,Täyttöaste,Vierasmatka,Kotivoitto
0,1,2014-09-10,18:30:00,Kärpät,Tappara,3,1,N,5608,3-1,4,2014-2015,1,0,0,9,Wednesday,0.847898,497,1
1,2,2014-09-11,18:30:00,HIFK,Blues,3,2,N,6413,3-2,5,2014-2015,1,0,0,9,Thursday,0.782073,21,1
2,3,2014-09-11,18:30:00,Lukko,Ässät,2,3,VL,4429,2-2,4,2014-2015,X,0,0,9,Thursday,0.820185,51,0
3,4,2014-09-12,18:30:00,Ilves,Tappara,1,2,N,6349,1-2,3,2014-2015,2,0,0,9,Friday,0.869726,0,0
4,5,2014-09-12,18:30:00,JYP,Sport,4,2,N,3843,4-2,6,2014-2015,1,0,0,9,Friday,0.866126,272,1


In [234]:
X = df[['Kotijoukkue','Vierasjoukkue','Koti_pelasi_eilen','Vieras_pelasi_eilen','Kuukausi','Viikonpäivä','Täyttöaste','Vierasmatka']]
y = df.Kotivoitto

In [187]:
X.head()

,Kotijoukkue,Vierasjoukkue,Koti_pelasi_eilen,Vieras_pelasi_eilen,Kuukausi,Viikonpäivä,Täyttöaste,Vierasmatka
0,Kärpät,Tappara,0,0,9,Wednesday,0.847898,497
1,HIFK,Blues,0,0,9,Thursday,0.782073,21
2,Lukko,Ässät,0,0,9,Thursday,0.820185,51
3,Ilves,Tappara,0,0,9,Friday,0.869726,0
4,JYP,Sport,0,0,9,Friday,0.866126,272


Seuraavaksi muutetaan kategoriset sarakkeet kotijoukkue, vierasjoukkue, kuukausi ja viikonpäivä ns. Dummy-muuttujiksi, jotta data olisi koneoppimismallille optimaalisessa muodossa. Tämän jälkeen pudotetaan pois alkuperäiset kategoriset muuttujat ja lisätään niiden tilalle luodut "dummy" muuttujat. Tämän voisi ehkä tehdä helpomminkin...

In [235]:
a = pd.get_dummies(df.Kotijoukkue,drop_first=True)
b = pd.get_dummies(df.Vierasjoukkue,drop_first=True)
c = pd.get_dummies(df.Kuukausi,drop_first=True)
d = pd.get_dummies(df.Viikonpäivä,drop_first=True)

In [236]:
e = X[['Koti_pelasi_eilen','Vieras_pelasi_eilen','Täyttöaste','Vierasmatka']]

In [237]:
frames = [a,b,c,d,e]

X = pd.concat(frames, axis=1)

In [223]:
X.head()

,HIFK,HPK,Ilves,JYP,Jukurit,KalPa,KooKoo,Kärpät,Lukko,Pelicans,SaiPa,Sport,TPS,Tappara,Ässät,HIFK,HPK,Ilves,JYP,Jukurit,KalPa,KooKoo,Kärpät,Lukko,Pelicans,SaiPa,Sport,TPS,Tappara,Ässät,2,3,9,10,11,12,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Koti_pelasi_eilen,Vieras_pelasi_eilen,Täyttöaste,Vierasmatka
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0.847898,497
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0.782073,21
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0.820185,51
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.869726,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.866126,272


In [270]:
#Train-test split
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [239]:
from sklearn.preprocessing import StandardScaler

In [271]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Deep-learning

In [280]:
model = tf.keras.models.Sequential()

#Input kerros
model.add(tf.keras.layers.Dense(50,activation = 'relu', input_dim = 46))

# 1st hidden layer
model.add(tf.keras.layers.Dense(50,activation = 'relu'))

# 2nd hidden layer
model.add(tf.keras.layers.Dense(50,activation = 'relu'))

#Output kerros
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [281]:
model.fit(x=X_train, y=y_train, batch_size=10, epochs=100)

Epoch 1/100
1327/1327 [==============================] - 2s 1ms/step - loss: 0.6886 - acc: 0.5365
Epoch 2/100
1327/1327 [==============================] - 1s 502us/step - loss: 0.6536 - acc: 0.6112
Epoch 3/100
1327/1327 [==============================] - 1s 436us/step - loss: 0.6293 - acc: 0.6458
Epoch 4/100
1327/1327 [==============================] - 1s 389us/step - loss: 0.5983 - acc: 0.6737
Epoch 5/100
1327/1327 [==============================] - 0s 365us/step - loss: 0.5723 - acc: 0.7091
Epoch 6/100
1327/1327 [==============================] - 1s 484us/step - loss: 0.5424 - acc: 0.7242
Epoch 7/100
1327/1327 [==============================] - 1s 600us/step - loss: 0.5105 - acc: 0.7634
Epoch 8/100
1327/1327 [==============================] - 1s 605us/step - loss: 0.4756 - acc: 0.7792
Epoch 9/100
1327/1327 [==============================] - 1s 578us/step - loss: 0.4397 - acc: 0.8041
Epoch 10/100
1327/1327 [==============================] - 1s 552us/step - loss: 0.3964 - acc: 0.8214
E

1327/1327 [==============================] - 1s 910us/step - loss: 0.0484 - acc: 0.9797 0s - loss: 0.0398 - acc: 0.984 - ETA: 1s - loss: 0
Epoch 77/100
1327/1327 [==============================] - 1s 900us/step - loss: 0.0314 - acc: 0.9842
Epoch 78/100
1327/1327 [==============================] - 1s 577us/step - loss: 0.0470 - acc: 0.9812
Epoch 79/100
1327/1327 [==============================] - 1s 527us/step - loss: 0.0449 - acc: 0.9789
Epoch 80/100
1327/1327 [==============================] - 1s 837us/step - loss: 0.0363 - acc: 0.9842
Epoch 81/100
1327/1327 [==============================] - 1s 708us/step - loss: 0.0375 - acc: 0.9834
Epoch 82/100
1327/1327 [==============================] - 1s 528us/step - loss: 0.0369 - acc: 0.9781 2s - loss: 0.0097 - 
Epoch 83/100
1327/1327 [==============================] - 1s 505us/step - loss: 0.0441 - acc: 0.9781
Epoch 84/100
1327/1327 [==============================] - 1s 860us/step - loss: 0.0408 - acc: 0.9797 0s - loss: 0.0351 - acc: 0.9
Epo

In [282]:
y_preds = model.predict(X_test)

In [283]:
y_preds = y_preds > 0.5

In [278]:
from sklearn.metrics import classification_report, confusion_matrix

In [284]:
print(classification_report(y_test,y_preds))

             precision    recall  f1-score   support

          0       0.62      0.59      0.61       252
          1       0.50      0.53      0.51       191

avg / total       0.57      0.56      0.57       443



In [285]:
confusion_matrix(y_preds, y_test)

array([[149,  90],
       [103, 101]], dtype=int64)

In [218]:
df.Kotivoitto.value_counts(normalize=True)

0    0.550847
1    0.449153
Name: Kotivoitto, dtype: float64

Deep - lerning malli pääsi tässä tapauksessa 57% tarkkuuteen. En odottanut tuloksena mitään järin tarkkaa mallia, luonnollisestikaan, odotukset olivat noin 60%-tasolla johon melko lähelle päästiinkin. Verrataan esimerkiksi malliin, joka ennustaisi kaikki ottelut kotivoitoksi, niin se pääsisi n. 45% tarkkuuteen tai malliin, joka ennustaisi kaikki ottelut ei-kotivoitoksi pääsisi 55% tarkkuuteen. Myös tämän harjoitteen kautta huomataan, että jääkiekko on varsin vaikeasti ennustettava laji, ja erityisesti sm-liiga, joka on erityisen tasainen sarja. Mallia voitanee kehittää vielä eteenpäin, nimetäänkin tämä v.1.0:ksi ja palataan (ehkä) sorvin ääreen myöhemmin.  